In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Load the data and overviewit

In [ ]:
#load data
World_report_2021 = pd.read_csv("world-happiness-report-2021.csv")
# #overwiew the data
World_report_2021.info()
# any data is missing
World_report_2021.isnull().sum()
World_report_2021.head()

Where is the happy country

In [ ]:
#sort by Ladder score, Finland is first and Afghanistan is last
World_report_2021=World_report_2021.sort_values(by=["Ladder score"],ascending=False)


 Country name     Regional indicator     Ladder score  
      Finland     Western Europe         7.842   
      Denmark     Western Europe         7.620   
  Switzerland     Western Europe         7.571   
      Iceland     Western Europe         7.554   
  Netherlands     Western Europe         7.464 

In [ ]:
#create graph with top 10 and bottom 10
top10_country=World_report_2021.groupby("Country name")["Ladder score"].mean().sort_values(ascending=False)[:10]
bottom10_country=World_report_2021.groupby("Country name")["Ladder score"].mean().sort_values(ascending=False)[-10:]
TB10_country=top10_country.append(bottom10_country).reset_index()
plt.figure(figsize=(20,7))
sns.barplot(y="Country name",x="Ladder score",data=TB10_country,palette="Spectral" )
for x,y in enumerate(TB10_country["Ladder score"]):
    plt.text(y+0.05,x,str(float(y)))
sns.despine()
# plt.show()

<a href="https://sm.ms/image/VYadrKMm6JoWsPz" target="_blank"><img src="https://i.loli.net/2021/04/18/VYadrKMm6JoWsPz.png" ></a>

Whats the differet in happy and unhappy country

In [ ]:
#Separate the happy country or not
LS_median= World_report_2021["Ladder score"].median() #median in ladder score
World_report_2021["Happy_country"]=World_report_2021["Ladder score"].apply(lambda x:1 if x >=LS_median else 0)
happy=World_report_2021[World_report_2021["Happy_country"] ==1]
unhappy = World_report_2021[World_report_2021["Happy_country"] ==0]

In [ ]:
#create the graph 
columns_list=['Logged GDP per capita','Social support',
                'Healthy life expectancy','Freedom to make life choices',
                'Generosity','Perceptions of corruption']
for col in columns_list:
    fig=sns.kdeplot(happy[col], shade=True,color="r",alpha=0.3)
    fig=sns.kdeplot(unhappy[col], shade=True,color="g",alpha=0.3)
    fig.get_yaxis().set_visible(False)
    for s in ["top", "right", "left"]:
        fig.spines[s].set_visible(False)
    plt.title("Happy country in red, unhappy country in green")
#     plt.show()

<a href="https://sm.ms/image/BDJE9FjdQ1XINUg" target="_blank"><img src="https://i.loli.net/2021/04/18/BDJE9FjdQ1XINUg.png" ></a><a href="https://sm.ms/image/oKx4e9RQ3W8FbS6" target="_blank"><img src="https://i.loli.net/2021/04/18/oKx4e9RQ3W8FbS6.png" ></a><a href="https://sm.ms/image/BOH79qfyIo23wka" target="_blank"><img src="https://i.loli.net/2021/04/18/BOH79qfyIo23wka.png" ></a><a href="https://sm.ms/image/76NbmuL94JzpXoh" target="_blank"><img src="https://i.loli.net/2021/04/18/76NbmuL94JzpXoh.png" ></a><a href="https://sm.ms/image/AcgQDnVfp8UsWiN" target="_blank"><img src="https://i.loli.net/2021/04/18/AcgQDnVfp8UsWiN.png" ></a><a href="https://sm.ms/image/ASDPWR5rIoa3dEs" target="_blank"><img src="https://i.loli.net/2021/04/18/ASDPWR5rIoa3dEs.png" ></a>

Create map to clear where the happy country is

In [ ]:
#Clean data
country_code = pc.map_country_name_to_country_alpha3()#set the country code
World_report_name=World_report_2021.set_index("Country name")
temp_data = pd.DataFrame(World_report_name['Ladder score']).reset_index() #only need the country name and ladder score
#found the country not in ISO3166, could be the name is different
Not_in_ISO3166=temp_data[temp_data["Country name"].isin(country_code)==False]  #6 is not in list
#Change the name to country code or delete
temp_data.loc[temp_data["Country name"] == "Taiwan Province of China", "Country name"]= "Taiwan, Province of China"
temp_data.loc[temp_data["Country name"] == "Hong Kong S.A.R. of China", "Country name"]= "Hong Kong"
temp_data.loc[temp_data["Country name"] == "Congo (Brazzaville)", "Country name"]= "Congo"
temp_data.loc[temp_data["Country name"] == "Palestinian Territories", "Country name"]= "Palestine, State of"
temp_data.drop(temp_data[temp_data["Country name"]=="Kosovo"].index,inplace=True) #Not part of the ISO 3166 standard
temp_data.drop(temp_data[temp_data["Country name"]=="North Cyprus"].index,inplace=True) #Not part of the ISO 3166 standard
temp_data['iso_alpha'] = temp_data['Country name'].apply(lambda x:pc.country_name_to_country_alpha3(x,))

In [ ]:
#create graph
fig=px.choropleth(temp_data, locations='iso_alpha',color='Ladder score',hover_name='Country name',color_continuous_scale=px.colors.diverging.Fall)
fig.update_layout(title_text='World map  Ladder score',geo_showframe=False,title_font_size=40)
# fig.show()

<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plotly.com/~iilagger/4.embed"></iframe>

Compare in different region

In [ ]:
#clean data with country name, region and ladder scoure
new_report=World_report_2021[["Country name","Regional indicator","Ladder score"]]
#ladder score in different country
# print(new_report.head())
fig = sns.kdeplot(new_report["Ladder score"],hue=new_report["Regional indicator"],fill=True)
fig.get_yaxis().set_visible(False)
for i in ["top", "right", "left"]:
        fig.spines[i].set_visible(False)
# plt.show()

<a href="https://sm.ms/image/ykxqNhjMaXAYcse" target="_blank"><img src="https://i.loli.net/2021/04/18/ykxqNhjMaXAYcse.png" ></a>

Who is the most happy country in that region

In [ ]:
#country in Sub-Saharan Africa
Sub_Saharan_Africa=new_report.loc[new_report["Regional indicator"]=="Sub-Saharan Africa"].sort_values(by=["Ladder score"],ascending=False).reset_index(drop=True).loc[0]
#'Western Europe'
Western_Europe =new_report.loc[new_report["Regional indicator"]=="Western Europe"].sort_values(by=["Ladder score"],ascending=False).reset_index(drop=True).loc[0]
# 'North America and ANZ'
North_America_and_ANZ =new_report.loc[new_report["Regional indicator"]=="North America and ANZ"].sort_values(by=["Ladder score"],ascending=False).reset_index(drop=True).loc[0]
# 'Middle East and North Africa'
Middle_East_and_North_Africa =new_report.loc[new_report["Regional indicator"]=="Middle East and North Africa"].sort_values(by=["Ladder score"],ascending=False).reset_index(drop=True).loc[0]
#  'Latin America and Caribbean'
Latin_America_and_Caribbean =new_report.loc[new_report["Regional indicator"]=="Latin America and Caribbean"].sort_values(by=["Ladder score"],ascending=False).reset_index(drop=True).loc[0]
#  'Central and Eastern Europe'
Central_and_Eastern_Europe =new_report.loc[new_report["Regional indicator"]=="Central and Eastern Europe"].sort_values(by=["Ladder score"],ascending=False).reset_index(drop=True).loc[0]
#  'East Asia'
East_Asia =new_report.loc[new_report["Regional indicator"]=="East Asia"].sort_values(by=["Ladder score"],ascending=False).reset_index(drop=True).loc[0]
#  'Southeast Asia'
Southeast_Asia =new_report.loc[new_report["Regional indicator"]=="Southeast Asia"].sort_values(by=["Ladder score"],ascending=False).reset_index(drop=True).loc[0]
#  'Commonwealth of Independent States'
Commonwealth_of_Independent_States =new_report.loc[new_report["Regional indicator"]=="Commonwealth of Independent States"].sort_values(by=["Ladder score"],ascending=False).reset_index(drop=True).loc[0]
#  'South Asia'
South_Asia=new_report.loc[new_report["Regional indicator"]=="South Asia"].sort_values(by=["Ladder score"],ascending=False).reset_index(drop=True).loc[0]
#merge all the data
Merge_data=pd.DataFrame([South_Asia,Commonwealth_of_Independent_States,
                         East_Asia,Central_and_Eastern_Europe,
                         Latin_America_and_Caribbean,Middle_East_and_North_Africa,
                         North_America_and_ANZ,Western_Europe,
                         Sub_Saharan_Africa]).sort_values(by=["Ladder score"],ascending=False)

In [ ]:
#               Country name                  Regional indicator  Ladder score
# 0                   Finland                      Western Europe         7.842
# 0               New Zealand               North America and ANZ         7.277
# 0                    Israel        Middle East and North Africa         7.157
# 0                Costa Rica         Latin America and Caribbean         7.069
# 0            Czech Republic          Central and Eastern Europe         6.965
# 0  Taiwan Province of China                           East Asia         6.584
# 0                Uzbekistan  Commonwealth of Independent States         6.179
# 0                 Mauritius                  Sub-Saharan Africa         6.049
# 0                     Nepal                          South Asia         5.269